# Neural Networks and Deep Learning

A simple regression function takes size of a house as an input and returns price. In the example, a rectified linear unit was given. RELU is the acronym here.

In a more complex example, it is suggested that by adding a layer, the network can be trained to detect relevant features. For example, in housing price the hidden layers might become sensitive to school quality, family size and walkability, all of which affect housing price. All the neurons are connected to the input layer but the weighting will determine which neurons affect the hidden layer activation. Of course, with enough data, the network is trained, so we don't have to engineer the features read in the hidden layers.

> To use a topic of interest to me, a home run predictor might have a hidden layer that detects launch angle, velocity, field direction, and ball park. These would typically be sufficient to detect whether a batted ball will be a homerun.

Note that the meaning of the features will may be difficult to interpret. Using my HR example, you _may_ find that launch angle is an implicit feature. But you may _not_ find it. That is, of course, part of the point: NNs allow us to "design" systems without designing the features they need to work well.

## Network Types

A _standard neural network_ might be used in online advertising, real estate pricing. 

A _convollutional network (CNN)_ might be used in image recognition.

A _recurrent neural network (RNN)_ might be use with language or other sequential data. 

## Structured and Unstructured Data

They just give examples, but structured data is data that has been recorded and usually takes discrete or continuous values such as housing prices or whether something is a cat; unstructured data is data that's not been so classified, such as an image. 

## Scale drives NN

Neural Network outperform other learning models when (1) we have lots of data and (2) to the degree that the network is sophisticated.

There's a graph of network performance that looks like logarithmic functions. In the region of limited amounts of data, the performance ordering is not well defined. in that region, feature engineering tends to be of the essence in determining algorithm performance.

### Algorithmic advancements are also important

Algorithms:

1. improve computational speed, for example RELU is way faster in gradient descent than sigmoid. 
1. _It sounded like there was going to be a list of reasons why algorithms improve the process, over and above the data/computational changes that have accelerated deep learning._


## Logisitic Regression

We want a function that expresses the probability of y = 1 given x, $\hat{y} = P(y = 1 | x)$ where $0 \le \hat{y} \le 1$. 
$$
\hat{y} = \sigma(w^{T} + b)
$$
$/sigma$ is the sigmoid function. 
$$
\sigma(z) = \frac{1}{1 + e ^ {-z}}
$$

### Cost Function

The phrase loss function and cost function seem to be used interchangably. (_Nope. But good eye. See below._)Anyway, you might thing we want something like 
$$
L(\hat{y},y) = \frac{1}{2}(\hat{y}-y)^{2}
$$
but that makes for a poor choice with gradient descent. (More on GD later.)

In order to get a convex optimization problem, we like:
$$
L(\hat{y},y) = -(y \textrm{log} \hat{y} + (1-y)\textrm{log}(1-\hat{y}))
$$

Hah! The loss function is defined for a single example; the cost function is the mean loss for all examples.
$$
J(w,b) = \frac{1}{m}\sum\limits_{i=1}^m L(\hat{y}^{(i)},y^{(i)})
$$


## Gradient Descent

The cost function defines a convex hypersurface. Suppose we are trying to minimize $J(w)$. Each step of gradient descent updates $w$ according to the following rule:
$$
w := w - \alpha \frac{dJ(w)}{dw}
$$

Where $\alpha$ is the learning rate and these are deriviatives.

This makes sense when you think about it. Our derivative represents a rate of change, which will be zero at the minimum of the hypersurface; thus, at that minimum, $w$ updates to $w$, and elsewhere, it moves in the direction of the minimum.

Of course, what we're actually trying to optimize are $w$ and $b$ and the functions we care about are...
$$
w := w - \alpha \frac{\partial J(w,b)}{\partial w}
$$

and 

$$

b := b - \alpha \frac{\partial J(w,b)}{\partial b}

$$

Latex appears to be broken.

## Computation Graphs

SUppose you have a function, J(a,b,c) = 3(a + bc). We can think of this function in three steps:
$$
u = bc \\
v = a + u \\
J = 3v
$$
We can then ask what various derivatives are. E.g., what is $\frac{dJ}{dv}$? It's 3. 

We can compute others by the _chain rule_, which says
$$
\frac{dx}{dz} = \frac{dx}{dy}\frac{dy}{dz}
$$

When we write code, we're usually interested in a final output variable. By convention, we name derivatives like $\frac{dJ}{dv}$ ```dv```.

## Logistic Regression Derivatives

We can use computation graphs to handle the backward propagation step.


$\hat{y} = \sigma(w^{T} + b)$

a = $\sigma(z) = \frac{1}{1 + e ^ {-z}}$

$L(\hat{y},y) = -(y \textrm{log} \hat{y} + (1-y)\textrm{log}(1-\hat{y}))$

We compute the loss on a single example with the final equations. Thus, the first derivative we want is $\frac{dL(\hat{y},y)}{da}$.

What we are looking for is how much to change $w_1, w_2 \ldots w_n$, our logistic regression weights. It turns out that the relevant amounts are found with $x_i\frac{dL}{d\hat{y}}$:
$$
w_i := w_i - \alpha \frac{dL}{dw_i} = w_i - \alpha x_i\frac{dL}{d\hat{y}}
$$

In order to find the cost (cf. loss), we need to take the mean loss for our $m$ training examples. Logically, this means some ```for``` loops with each of our examples to find the update amounts, as above, and then taking the average. However, this is equivalent to a much faster vectorized process.

In [3]:
import numpy as np
a = np.array([1,2,3,4])
a

array([1, 2, 3, 4])

In [4]:
import time

a = np.random.rand(1_000_000)
b = np.random.rand(1_000_000)

tic = time.time()
c = np.dot(a,b)
toc = time.time()

print("Vect version: "+ str((toc-tic)*1000) + "ms")

c = 0
tic = time.time()
for i in range(1_000_000):
    c += a[i]*b[i]
toc = time.time()
print("Loop version: "+ str((toc-tic)*1000) + "ms")


Vect version: 114.34412002563477ms
Loop version: 964.1385078430176ms


## Vectorization

Vectorization uses SIMD (Single Instruction Mutliple Data) rather than a for loop. Numpy is, of course, full of elementwise operations for vectors:
```
np.log(v) ## elementwise log for values in v.
np.abs(v) ##abs value
np.maximum(v,0) #the larger of v and 0
v**2 ##square
v+1 ##increment v
etc.
```
Consider a block of code for finding cost:
```
##This is obviously not real code.

J = 0, dw1 = 0, dw2 = 0, ..., db = 0
for i in range(1,m):
    ##calculate each instance
    z_i = w.T_i + b
    a_i = sigmoid(z_i)
    J += -(y_i * log(a_i) + (1 - y_i)log(1-a_i) ##loss function
    dz_i = a_i - y_i
    for i in [dw1, dw2 ...]:
        dw_i += x[i]*dzi
J = J/m, dw1 = dw1/m, dw2 = dw2/m ##find their means
```

We can vectorize this an eliminate a for loop by replacing explicit ```dw1 = 0, dw2 = 0,....``` with ```dw = np.zeros``` and eliminating the final for loop by ```dw += dzi```


## Vectorizing Logistic Regression

For each training example, we need to find $z_i = w_i^Tx_i+b$.

To do that, we find $Z = w^TX + \hat{b}$, where $\hat{b}$ is just a row vector length $m$ of $b$s.

Then we find the sigmoid of $Z$.

In [1]:
##This is the step in python, using an arbitrary example.

X = np.array([0,1,2,3,4,5,6,7,8]).reshape(3,3)
w = np.array([2,3,4])
b = 2
Z = np.dot(w.T,X
          ) + b
Z

##The sigmoid calculation is left as an exercise for the project.

NameError: name 'np' is not defined

### Vectorizing the Gradient

To vectorize the gradient, we're going to find

## Notes on Numpy objects

numpy objects that look like row vectors may not be, and they will misbehave if used like one. 

In [ ]:
## this is not a real vector
print("A vector-looking thing")
not_v = np.arange(6)
print(not_v)

## we can see this by checking the shape.
print(not_v.shape)

## this is a real vector; it's a row vector:
print("A real vecor")
v = np.arange(6).reshape(1,6)
print(v)
print(v.shape)

#Notice the behavior when we take the transpose.
print("Their transposes")
print(not_v.T) 
print(v.T)



## Explaining the cost fucntion

If $y = 1$, $p(y|x) = \hat{y}$

If $y = 0$, $p(y|x) = 1 - \hat{y}$

These are both true if and only if:
$$p(y|x) = \hat{y}^y(1 - \hat{y})^{1 - y}$$

(It's fun to notice why that's correct by noting what happens when y is 0 and when it's 1.)

In [ ]:
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)
c = a*b
c

In [13]:
a = np.random.randn(3,3,3,1)
b = x.reshape(x.shape[0],-1)
a[0]

array([[[-0.25467577],
        [ 1.42151296],
        [ 0.4017047 ]],

       [[-0.19145451],
        [ 1.47867666],
        [ 0.0704399 ]],

       [[ 0.30746377],
        [-0.79876817],
        [ 0.09971697]]])

# Neural Network Representation

We have an input layer, denoted $a^{[0]}$; previously we called this $X = {x_1,x_2,... x_n}$.

Our next layer is a hidden layer, denoted $a^{[1]} = a^{[1]}_1,...a^{[1]}_n$

There may be additional layers, but the final layer is the output layer, which is a real number value which equals $\hat{y}$. 

Note that this is a two layer network (if it has one hidden layer and an output layer). The inputs aren't neurons, I guess, so that's why it has such a name.

Associated with each layer are parameters $w, b$ with suitable superscripts (e.g., $w^{[1]}, b^{[1]}$). $w^{[1]}$ with have a number of rows equal to the number of neurons in $a^{[1]}$ and a number of columns equal to the number of features in $a^{[0]}$; $b^{[1]}$ is column vector with rows for each neuron in $a^{[1]}$ .

In general, the parameters associated with each layer are derived from the number of neuron in the layer and the number of neurons that are inputs to it. The number of rows equals the layers number of neurons, the number of columns equals the number of input neurons.

Note that a logistic regression is basically a single layer network with a single neuron in the output layer.


## Computing the Output

Each node computes a linear weighting of the input values and then computes the activation function (which we're still assuming is the sigmoid.)

Thus $a^{[l]}_i$ computes $z^{[l]}_i = w^{[l]T}_i x + b^{[l]}_i$ and computes $\sigma (z^{[l]}_i)$

Not surprisingly, we're going to stack all this together to vectorize the process. This gives us two big, neat matrixy equations:

$$
z^{[1]} = W^{[1]}x + b^{[1]}\\
a^{[1]} = \sigma(z^{[1]})
$$


## Activation Functions

The tanh function:
$$a = tanh(z) = \frac{e^{z}-e^{-z}}{e^{z} + e^{-z}}$$
This has the effect of "centering" the activation function around 0 so it provides a sort of normalization. It otherwise is just a permutation of the sigmoid. It is almost always better than sigmoid, however. The one notable exception is that in the final output layer for a binary classification, it makes sense for the output to be in a range of 0 to 1. So, in the hidden layers, we would usually prefer tanh but perhaps prefer sigmoid in the final layer.

The rectified linear unit (ReLU):
$$
ReLU(z) = max(0,z)\\
\partial ReLU(z)  = 0 \textrm{ if }  z < 0; 1 \textrm{ if } z > 0
$$

Leaky ReLU. This is fun. It looks like a simple hack to solve the uniform zero derivative where $z < 0$. You just give it a small positive slope.

You can choose the slope for z < 1; something small like:
$$
\textrm{leaky ReLU}(z) = max(0.01,z)\\
$$

**ReLU has become the default.** However, there are problems where other functions might be superior. See [this simulator](http://playground.tensorflow.org); the spiral problem is most easily solved with tanh; try it and see if you don't believe me. (BTW, you find a pretty good solution for the spiral with a two hidden layer network containing 8 neurons and 2 neurons. Include X1 and X2 and their sines as the features.  Learning rate = .03.) 


## Derivatives of the activation functions
For the sigmoid function
$$
\frac{d}{dz}\sigma(z) = \sigma(z)(1 - \sigma(z)) 
$$

For tanh
$$
\frac{d}{dz}tanh(z) = 1 - tanh^2(z)
$$

See above for ReLU.

Note that ReLU is not defined for $z = 0$. The probability that you're going to stumble on z=0 and throw and exception in your code is tiny. A solution is just to set this point's derivative to either 1 or 0 to avoid the error.

The propagation equations

Forward, you pretty much know these:
$Z^{[1]} = W^{[1]}X + b^{[1]}$

$A^{[1]} = g^{[1]}(Z^{[1]})$

As so on for the rest of the layers, which take the output of the previous layer in place of X.

Backward:

$dZ^{[2]} = A^{[2]} - Y$

$ dW^{[2]} = \frac{1}{m} dZ^{[2]}A^{[1]T}$

```
db^{[2]} = \frac{1}{m}np.sum(dZ^{[2]}, axis = 1, keepdims = True)
##this is in code because we're summing rows of a matrix
```

$dZ^{[1]} = W^{[2]T}dZ^{[2]} \times g^{[1]}(Z^{[1]})$
(This $\times$ indicates elementwise multiplication.

$dW^{[1]} = \frac{1}{m}dZ^{[1]}X^T$

```db^{[1]} = 1/m*np.sum(dZ[1],axis = 1, keepdims = True)```


## Random Initialization

If you don't initialize randomly, every neuron calculates the same function and has the same derivative so your network reduces to a single neuron. We initialize randomly to avoid this. We usually multiply our random numbers by a small constant because if the numbers are larger, learning is slower (look at the derivaties, ~=0, at the extremes of tanh.)

# Deep Networks

Networks are typically called "deep" when the number of layers is greater than 2, i.e., when there are multiple hidden layers prior to the output layer.

### Notational Stuff

$L$ denotes the number of layers 

$n^{[l]}$ units in layer l

$a^{[l]}$ activations in l

$a^{[l]} = g^{[l]}(z^{[l]})$ by the way

$w^{[l]}$ weights for $z^{[l]}$


## Forward propagation in a deep NN

$z^{[l]} = W^{[l]}a^{[l-1]}+b^{[l]}$

$a = g^{[l]}(z^{[l]})$

We vectorize in the expected way (or at least, the way I expected). Note that we will need a for loop to complete the propagation for each layer; that cannot be vectorized.

## Thinking about the dimensions

The output of each layer, $z^{[l]}$, is the result of $W^{[l]}a^{[l-1]} + b^{[l]}$. The dimensions of the output layer are thus $(n^{[l]},1)$ and the input layer dimensions are $(n^{[l-1]},1)$. $W^{[l]}$ must therefore have dimensions $(n^{[l]},n^{[l-1]})$. $b$ is obviously a colmn vector with the dimensions of $(n^{[l]},1)$.

When we vectorize, the 1's are replaced with m's, because we have m training examples. 

Ng says to use this to help debug code. 

## Why deep networks?


### "Feature Construction" explanation
The first hidden layer of a network constructs features of the input layer. These are passed as inputs to the next layer. These features are then inputs to constuct a next level of features, typically of a higher level of complexity, from the features constructed at the prior level. At each layer, we get a little more complexity in the construction.

The example given is face recognition in a three layer network (the example is slightly artificial in the complexity of the network.) Layer one detects edges, layer two detects facial components (eyes nose, mouth, chin, etc.), layer three detects faces (which are composed of facial components.) 

Here's a baseball example (since I like those): suppose you have a bunch of hittrack data. You will have angles and speeds in the input features. At a next level you might generate x, y and z velocities. At the third level, distances traveled, landing position, etc. The final level could classify the batted ball as a home run, fly ball out or other type of batted ball.

### Circuit Theory explanation

Informally, this says that there are functions that can be computed in a deep network that would require an exponentially larger network to compute with a shallow network.

XOR provides an example. You need just log(n) hidden units to computer XOR for n features using a deep network. To do the same with a shallow network of one layer, you need $2^n$ units.

### Branding

I love this: he just admits that the phrase "deep learning" is a recentish brand of an existing model and it's taken off because it sounds sexy. That's really an explanation for why we talk so much about deep learning, not an explanation of why it works. But it's funny to note it because 

## Tuning

The parameters of the model are $W^{[1]}$, $b^{[1]}$, $W^{[2]}$, etc. These are tuned by the forward and backward propagation.

The hyperparameters include other values that we need to specify. Tuning our network generally requires tuning our hyperparameters. Our hyperparameters include:
- learning rate
- number of iterations.
- num. of hidden layers
- num of layer units
- choice of activation function.

Later we will learn about a few more, like momentum, regularization, batch size.


**"Applied deep learning is a very empirical process"**

1. There's a fair amount of trial and error involved, iteration and refining.
1. Developing intuitions is a matter of experience.
1. Experiences in one domain may or may not translate to another.
1. Even developed networks may find that re-tuning some parameters is valuable after time: changes to data, changes in hardware, etc. can all allow for further tuning.